**2022-1 ML 과제**

작성자 : 김상회

Machine Learning 과제는 이전 교육세션에서 배운 "FE"와 "ML"을 사용하여, 타이타닉 생존 여부를 예측하는 분류 문제에서 직접 모델링 하고 제출까지 해보는 과제 입니다.  
  
코드는 기초적인 전처리와 Decision Tree를 사용하여 예측을 진행합니다. 필수는 아니지만 주어진 코드 이외에도 다른 모델과 방법들을 자율적으로 사용하여 성능을 높여보시기를 추천드립니다.  
  
과제하시면서 질문 있으신 분들은 언제든 질문방 or 개인톡으로 연락주세요 :)  
+) Ctrl + F에서 "TODO"를 찾고 순차적으로 진행하세요

In [ ]:
import pandas as pd
import numpy as np

# 데이터셋 로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# colab 사용

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/YBIGTA_실습용/ML/train_titanic.csv', index_col=0)
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/YBIGTA_실습용/ML/test_titanic.csv', index_col=0)
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/YBIGTA_실습용/ML/submission.csv')

In [ ]:
# local 사용

# train = pd.read_csv('train_titanic.csv')
# test = pd.read_csv('test_titanic.csv')
# submission = pd.read_csv('submission.csv')

In [ ]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


# 간단한 전처리

## 결측치 Drop  
결측치가 있는 column은 drop하도록 하겠습니다. 성능을 높이기 위해 drop하지 않고 다른 방법을 사용할 수 있습니다.  
ex) Embarked는 두명만 결측치인데 해당 값을 채워주는 방식

In [ ]:
#결측치의 수
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
train_dropna = train.drop(columns=["Age", "Cabin", "Embarked"])

## 카테고리 변수  

In [ ]:
# object 변수 찾기
train_dropna.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
dtype: object

In [ ]:
# object 변수 안에 몇개의 값이 존재하는 지 -> nunique() 사용
train_dropna[["Name", "Sex", "Ticket"]].nunique()

Name      891
Sex         2
Ticket    681
dtype: int64

In [ ]:
# Name과 Ticket은 수가 많기 때문에 drop하고 Sex는 남녀 두개이기 때문에 더미화 진행합니다.
train_object = train_dropna.drop(columns=["Name", "Ticket"])
train_object = pd.get_dummies(train_object, columns=["Sex"]) # 더미화 진행 -> pd.get_dummiex()
train_object.head()

,PassengerId,Survived,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male
0,1,0,3,1,0,7.2500,0,1
1,2,1,1,1,0,71.2833,1,0
2,3,1,3,0,0,7.9250,1,0
3,4,1,1,1,0,53.1000,1,0
4,5,0,3,0,0,8.0500,0,1


# Train Valid Split  
전처리가 끝난 Train Set을 검증을 위한 Validation Set으로 나눠줍니다.

In [ ]:
# X와 y 설정
target = "Survived"
X = train_object.drop(columns=[target])
y = train_object[target]

In [ ]:
# Train Test Split
from sklearn.model_selection import train_test_split

# TODO 1)
# X와 y를 train valid로 split 하세요
# test size는 0.2, random state는 616, stratify는 y로 설정하세요

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 616, stratify = y)

In [ ]:
print(f"X_train shape : {X_train.shape}, X_valid shape : {X_valid.shape}")

X_train shape : (712, 7), X_valid shape : (179, 7)


In [ ]:
# train과 valid의 y값 비율이 같아야 함

print(y_train.value_counts(normalize=True))
print(y_valid.value_counts(normalize=True))

0    0.616573
1    0.383427
Name: Survived, dtype: float64
0    0.614525
1    0.385475
Name: Survived, dtype: float64


# 모델 학습  
기본 Decision Tree를 사용하여 생존 여부를 예측하고 Grid Search를 사용하였을 때와 Validation Set에서의 성능을 비교합니다.

## 기본 DT

In [ ]:
# 기본 Decision Tree 예측
from sklearn.tree import DecisionTreeClassifier

# 모델 선언
dt = DecisionTreeClassifier() 

# 모델 학습
# TODO 2) 모델을 train 데이터에 맞춰 학습(fit)시키세요
dt.fit(X_train, y_train)

# Validation Set에 예측
# TODO 3) 학습한 모델로 valid 값을 예측하세요
pred_valid = dt.predict(X_valid)

In [ ]:
# 모델 예측 성능을 검증하기 위해 confusion matrix와 Accuracy, f1 score를 확인합니다.

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report

# confusion matrix
matrix = confusion_matrix(y_valid, pred_valid)
print(matrix)

# Accuracy
acc = accuracy_score(y_valid, pred_valid)
print('Accuracy: {:.2f}'.format(acc))

# f1 score
f1 = f1_score(y_valid, pred_valid)
print('F1_Score: {:.2f}'.format(f1))

[[86 24]
 [18 51]]
Accuracy: 0.77
F1_Score: 0.71


## Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
grid_dt = DecisionTreeClassifier()

# TODO 4)
# 테스트 하고 싶은 parameter 값들을 parameter 별로 sequence 구조의 데이터(list, numpy array, etc...)에 넣어줍니다.
# 이 때 변수 이름은 꼭 모델 parameter 이름과 동일할 필요는 없습니다.

max_depth = [2, 3, 4, 5, 6, 7, 8]
max_leaf = [4, 5, 6, 7, 8, 9, 10, 12, 16, 20, 24, 28, 32]

#위의 세 parameter를 하나로 묶기 위해서 다음과 같은 Dictionary 형태로 만들어줍니다.
#여기서 key 값은 str 형태로, 정확하게 모델 parameter 이름과 동일해야 합니다.

parameter_grid = {'max_depth': max_depth,
                  'max_leaf_nodes': max_leaf}

In [ ]:
gs = GridSearchCV(estimator= grid_dt, param_grid= parameter_grid) 
gs.fit(X_train, y_train) 

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8],
                         'max_leaf_nodes': [4, 5, 6, 7, 8, 9, 10, 12, 16, 20,
                                            24, 28, 32]})

In [ ]:
print('GridSearch 최적 parameter: {}'.format(gs.best_params_),
      'GridSearch 최고 Validation Score: {:.3f}'.format(gs.best_score_), sep = '\n')

GridSearch 최적 parameter: {'max_depth': 3, 'max_leaf_nodes': 6}
GridSearch 최고 Validation Score: 0.798


In [ ]:
# Grid Search로 찾은 최적의 파라미터 값으로 모델 생성
# TODO 5)
dt_grid = DecisionTreeClassifier(max_depth= 3, max_leaf_nodes= 6) 

dt_grid.fit(X_train, y_train) 
pred_valid_grid = dt_grid.predict(X_valid)

In [ ]:
# 모델 예측 성능을 검증하기 위해 confusion matrix와 Accuracy, f1 score를 확인합니다.

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report

# confusion matrix
matrix = confusion_matrix(y_valid, pred_valid_grid)
print(matrix)

# Accuracy
acc = accuracy_score(y_valid, pred_valid_grid)
print('Accuracy: {:.2f}'.format(acc))

# f1 score
f1 = f1_score(y_valid, pred_valid_grid)
print('F1_Score: {:.2f}'.format(f1))

[[101   9]
 [ 19  50]]
Accuracy: 0.84
F1_Score: 0.78


# Test Data 예측

## 전처리  
예측을 진행하기에 앞서 train에서 진행한 전처리 과정을 동일하게 적용해야 합니다.  
전처리 방법이 달라질 경우 결과도 다르게 나옵니다!  
(전처리 과정을 pipeline으로 만들어서 진행하기도 함 --> "전처리 pipeline" 검색)

### 결측치 Drop  
Drop ["Age", "Cabin", "Embarked"] Columns

In [ ]:
test_dropna = test.drop(columns=["Age", "Cabin", "Embarked"])

### 카테고리 변수  

In [ ]:
# Name과 Ticket은 수가 많기 때문에 drop하고 Sex는 남녀 두개이기 때문에 더미화 진행합니다.
test_object = test_dropna.drop(columns=["Name", "Ticket"])
test_object = pd.get_dummies(test_object, columns=["Sex"]) # 더미화 진행 -> pd.get_dummiex()
test_object.head()

,PassengerId,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male
0,892,3,0,0,7.8292,0,1
1,893,3,1,0,7.0000,1,0
2,894,2,0,0,9.6875,0,1
3,895,3,0,0,8.6625,0,1
4,896,3,1,1,12.2875,1,0


### 추가 결측치 전처리  
Fare 항목의 결측치가 존재하므로 해당 결측치를 처리해줘야함 -> train의 데이터는 Fare 행이 존재하기 때문에 drop 불가 -> 평균값으로 대체

In [ ]:
test_object.isna().sum()

PassengerId    0
Pclass         0
SibSp          0
Parch          0
Fare           1
Sex_female     0
Sex_male       0
dtype: int64

In [ ]:
test_fillna = test_object.fillna(test_object.mean()) # fillna(df.mean())을 사용하여 결측치 채우기

## 예측

In [ ]:
pred = dt_grid.predict(test_fillna)

In [ ]:
submission['Survived'] = pred
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [ ]:
submission.to_csv('dt_submission.csv', index=False)

본인 깃헙 repository에 과제 코드와 제출한 화면 캡처본을 함께 업로드 해주세요.

# 성능 향상을 위해 추가 코드 


In [ ]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
train.shape

(891, 12)

In [ ]:
test.shape

(418, 11)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 90.5+ KB


In [ ]:
train.isnull().sum()/len(train) * 100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

In [ ]:
train1 = train.drop(columns = 'Cabin')
train1
# 'Cabin' 버림

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


In [ ]:
train1.isnull().sum()/len(train1) * 100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Embarked        0.224467
dtype: float64

In [ ]:
train2 = train1
train2 = train2.drop(train2[train2['Embarked'].isnull()].index)
# 'Embarked'에서 null값인 index들 버림 (2개)

In [ ]:
train2.isnull().sum()/len(train2) * 100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.910011
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Embarked        0.000000
dtype: float64

In [ ]:
train3 = train2
train3['Age'] = train3['Age'].fillna(train3["Age"].median())
train3
# "Age"의 Null 값들 median()으로 채움

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


In [ ]:
train3.isnull().sum()/len(train3)*100

PassengerId    0.0
Survived       0.0
Pclass         0.0
Name           0.0
Sex            0.0
Age            0.0
SibSp          0.0
Parch          0.0
Ticket         0.0
Fare           0.0
Embarked       0.0
dtype: float64

In [ ]:
train3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Name         889 non-null    object 
 4   Sex          889 non-null    object 
 5   Age          889 non-null    float64
 6   SibSp        889 non-null    int64  
 7   Parch        889 non-null    int64  
 8   Ticket       889 non-null    object 
 9   Fare         889 non-null    float64
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 83.3+ KB


In [ ]:
train4 = train3
train4 = train4.drop(columns = ['Name', 'Ticket'])
train4
# 'Name', 'Ticket' feature 버림

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,S
887,888,1,1,female,19.0,0,0,30.0000,S
888,889,0,3,female,28.0,1,2,23.4500,S
889,890,1,1,male,26.0,0,0,30.0000,C


In [ ]:
train4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Sex          889 non-null    object 
 4   Age          889 non-null    float64
 5   SibSp        889 non-null    int64  
 6   Parch        889 non-null    int64  
 7   Fare         889 non-null    float64
 8   Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(2)
memory usage: 69.5+ KB


In [ ]:
train5 = train4
train5['Sex'] = train5['Sex'].map({"male": 1, "female": 0})
train5

# Sex의 categorical data를 encoding함

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,S
1,2,1,1,0,38.0,1,0,71.2833,C
2,3,1,3,0,26.0,0,0,7.9250,S
3,4,1,1,0,35.0,1,0,53.1000,S
4,5,0,3,1,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.0,0,0,13.0000,S
887,888,1,1,0,19.0,0,0,30.0000,S
888,889,0,3,0,28.0,1,2,23.4500,S
889,890,1,1,1,26.0,0,0,30.0000,C


In [ ]:
train5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Sex          889 non-null    int64  
 4   Age          889 non-null    float64
 5   SibSp        889 non-null    int64  
 6   Parch        889 non-null    int64  
 7   Fare         889 non-null    float64
 8   Embarked     889 non-null    object 
dtypes: float64(2), int64(6), object(1)
memory usage: 69.5+ KB


In [ ]:
train6 = train5
train6.drop(columns = 'PassengerId')

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,S
887,1,1,0,19.0,0,0,30.0000,S
888,0,3,0,28.0,1,2,23.4500,S
889,1,1,1,26.0,0,0,30.0000,C


In [ ]:
train7 = train6
train7 = pd.get_dummies(train7, columns = ['Embarked'])
train7

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,1,22.0,1,0,7.2500,0,0,1
1,2,1,1,0,38.0,1,0,71.2833,1,0,0
2,3,1,3,0,26.0,0,0,7.9250,0,0,1
3,4,1,1,0,35.0,1,0,53.1000,0,0,1
4,5,0,3,1,35.0,0,0,8.0500,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.0,0,0,13.0000,0,0,1
887,888,1,1,0,19.0,0,0,30.0000,0,0,1
888,889,0,3,0,28.0,1,2,23.4500,0,0,1
889,890,1,1,1,26.0,0,0,30.0000,1,0,0


In [ ]:
train8 = train7
train8 = train8.drop(columns = ['PassengerId'])
train8

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,0,0,1
1,1,1,0,38.0,1,0,71.2833,1,0,0
2,1,3,0,26.0,0,0,7.9250,0,0,1
3,1,1,0,35.0,1,0,53.1000,0,0,1
4,0,3,1,35.0,0,0,8.0500,0,0,1
...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,0,0,1
887,1,1,0,19.0,0,0,30.0000,0,0,1
888,0,3,0,28.0,1,2,23.4500,0,0,1
889,1,1,1,26.0,0,0,30.0000,1,0,0


In [ ]:
train8.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    889 non-null    int64  
 1   Pclass      889 non-null    int64  
 2   Sex         889 non-null    int64  
 3   Age         889 non-null    float64
 4   SibSp       889 non-null    int64  
 5   Parch       889 non-null    int64  
 6   Fare        889 non-null    float64
 7   Embarked_C  889 non-null    uint8  
 8   Embarked_Q  889 non-null    uint8  
 9   Embarked_S  889 non-null    uint8  
dtypes: float64(2), int64(5), uint8(3)
memory usage: 58.2 KB


In [ ]:
X = train8.drop(columns = 'Survived')
y = train8['Survived']

In [ ]:
print(X.shape)
print(y.shape)

(889, 9)
(889,)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# 0.2할지 0.3할지

In [ ]:
y_train.value_counts() / len(y_train)

0    0.61744
1    0.38256
Name: Survived, dtype: float64

In [ ]:
y_val.value_counts() / len(y_val)

0    0.617978
1    0.382022
Name: Survived, dtype: float64

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 39.2+ KB


In [ ]:
test.isnull().sum()/len(test) * 100

PassengerId     0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            20.574163
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.239234
Cabin          78.229665
Embarked        0.000000
dtype: float64

In [ ]:
test1 = test.drop(columns = 'Cabin')
test1

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S
...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,S


In [ ]:
test1.isnull().sum()/len(test1) * 100

PassengerId     0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            20.574163
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.239234
Embarked        0.000000
dtype: float64

In [ ]:
test2 = test1
test2['Fare'] = test2['Fare'].fillna(test2['Fare'].median())

In [ ]:
test2.isnull().sum()/len(test2)*100

PassengerId     0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            20.574163
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Embarked        0.000000
dtype: float64

In [ ]:
test3 = test2
test3['Age'] = test3['Age'].fillna(test3['Age'].median())
test3

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S
...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,27.0,0,0,A.5. 3236,8.0500,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,S
416,1308,3,"Ware, Mr. Frederick",male,27.0,0,0,359309,8.0500,S


In [ ]:
test3.isnull().sum()/len(test3)*100

PassengerId    0.0
Pclass         0.0
Name           0.0
Sex            0.0
Age            0.0
SibSp          0.0
Parch          0.0
Ticket         0.0
Fare           0.0
Embarked       0.0
dtype: float64

In [ ]:
test3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 35.9+ KB


In [ ]:
test4 = test3
test4 = test4.drop(columns = ['Name', 'Ticket'])
test4

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...,...
413,1305,3,male,27.0,0,0,8.0500,S
414,1306,1,female,39.0,0,0,108.9000,C
415,1307,3,male,38.5,0,0,7.2500,S
416,1308,3,male,27.0,0,0,8.0500,S


In [ ]:
test4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    object 
 3   Age          418 non-null    float64
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Fare         418 non-null    float64
 7   Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 29.4+ KB


In [ ]:
test5 = test4
test5['Sex'] = test5['Sex'].map({"male": 1, "female": 0})
test5

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.5,0,0,7.8292,Q
1,893,3,0,47.0,1,0,7.0000,S
2,894,2,1,62.0,0,0,9.6875,Q
3,895,3,1,27.0,0,0,8.6625,S
4,896,3,0,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...,...
413,1305,3,1,27.0,0,0,8.0500,S
414,1306,1,0,39.0,0,0,108.9000,C
415,1307,3,1,38.5,0,0,7.2500,S
416,1308,3,1,27.0,0,0,8.0500,S


In [ ]:
test5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    int64  
 3   Age          418 non-null    float64
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Fare         418 non-null    float64
 7   Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(1)
memory usage: 29.4+ KB


In [ ]:
test6 = test5
test6 = test6.drop(columns = 'PassengerId')
test6


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,Q
1,3,0,47.0,1,0,7.0000,S
2,2,1,62.0,0,0,9.6875,Q
3,3,1,27.0,0,0,8.6625,S
4,3,0,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...
413,3,1,27.0,0,0,8.0500,S
414,1,0,39.0,0,0,108.9000,C
415,3,1,38.5,0,0,7.2500,S
416,3,1,27.0,0,0,8.0500,S


In [ ]:
test7 = test6
test7 = pd.get_dummies(test7, columns = ['Embarked'])
test7

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,3,1,34.5,0,0,7.8292,0,1,0
1,3,0,47.0,1,0,7.0000,0,0,1
2,2,1,62.0,0,0,9.6875,0,1,0
3,3,1,27.0,0,0,8.6625,0,0,1
4,3,0,22.0,1,1,12.2875,0,0,1
...,...,...,...,...,...,...,...,...,...
413,3,1,27.0,0,0,8.0500,0,0,1
414,1,0,39.0,0,0,108.9000,1,0,0
415,3,1,38.5,0,0,7.2500,0,0,1
416,3,1,27.0,0,0,8.0500,0,0,1


In [ ]:
test7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      418 non-null    int64  
 1   Sex         418 non-null    int64  
 2   Age         418 non-null    float64
 3   SibSp       418 non-null    int64  
 4   Parch       418 non-null    int64  
 5   Fare        418 non-null    float64
 6   Embarked_C  418 non-null    uint8  
 7   Embarked_Q  418 non-null    uint8  
 8   Embarked_S  418 non-null    uint8  
dtypes: float64(2), int64(4), uint8(3)
memory usage: 24.1 KB


In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
# from sklearn.model_selection import GridSearchCV

# dt = DecisionTreeClassifier()

# criterion = ['gini', 'entropy']
# max_depth = [2, 3, 4, 5, 6, 7, 8]
# max_leaf = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]

# parameter_grid = {'max_depth': max_depth,
#                   'max_leaf_nodes': max_leaf,
#                   'criterion': criterion}

# gs = GridSearchCV(estimator= dt, param_grid= parameter_grid, scoring= 'roc_auc')
# gs.fit(X_train, y_train)

# # 가장 성능 좋은 parameter들 확인
# gs.best_params_
      

In [ ]:
# from xgboost import XGBClassifier
# from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
# from sklearn.model_selection import GridSearchCV

# xgb = XGBClassifier()

# eta = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
# gamma = [0, 0.5, 1, 2, 3]
# max_depth = [4, 5, 6, 7, 8]
# subsample = [0.5, 0.7, 1]

# parameter_grid = {
#     'eta' : eta,
#     'gamma' : gamma,
#     'max_depth' : max_depth,
#     'subsample' : subsample
# }

# gs = GridSearchCV(estimator = xgb, param_grid = parameter_grid, scoring = 'roc_auc')
# gs.fit(X_train, y_train)

# gs.best_params_

In [ ]:
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
# from sklearn.model_selection import GridSearchCV

# gb = GradientBoostingClassifier()

# learning_rate = [0.001, 0.1, 0.01]
# n_estimators = [100, 500, 1000]
# max_depth = [3, 4, 5]
# min_samples_split = [2, 3, 4, 5]


# parameter_grid = {
#     'learning_rate' : learning_rate,
#     'n_estimators' : n_estimators,
#     'max_depth' : max_depth,
#     'min_samples_split' : min_samples_split
# }

# gs = GridSearchCV(estimator = gb, param_grid = parameter_grid, scoring = 'roc_auc')
# gs.fit(X_train, y_train)

# gs.best_params_

In [ ]:
# decision tree 성능 확인
dt_temp = DecisionTreeClassifier(criterion='entropy', max_depth= 4, max_leaf_nodes= 15)
dt_temp.fit(X_train, y_train)
prediction = dt_temp.predict(X_val)

print('AUC SCORE')
print(roc_auc_score(y_val, prediction))

AUC SCORE
0.7300802139037432


In [ ]:
# logistic regression 성능 확인
from sklearn.linear_model import LogisticRegression

logreg_temp = LogisticRegression(max_iter= 500, solver= 'lbfgs')
logreg_temp.fit(X_train, y_train)
prediction2 = logreg_temp.predict(X_val)

print('AUC SCORE')
print(roc_auc_score(y_val, prediction2))

AUC SCORE
0.7910427807486631


In [ ]:
# svm 성능 확인
from sklearn import svm

svm_temp = svm.SVC()
svm_temp.fit(X_train, y_train)
prediction3 = svm_temp.predict(X_val)

print('AUC SCORE')
print(roc_auc_score(y_val, prediction3))

AUC SCORE
0.5998663101604278


In [ ]:
# svm(linear) 성능 확인
from sklearn import svm

svml_temp = svm.SVC(kernel = 'linear')
svml_temp.fit(X_train, y_train)
prediction4 = svml_temp.predict(X_val)

print('AUC SCORE')
print(roc_auc_score(y_val, prediction4))

AUC SCORE
0.7434491978609625


In [ ]:
# xgb 성능 확인
from xgboost import XGBClassifier

xgb_temp = XGBClassifier(max_depth= 3, subsample= 0.5)
xgb_temp.fit(X_train, y_train)
prediction5 = xgb_temp.predict(X_val)

print('AUC SCORE')
print(roc_auc_score(y_val, prediction5))

AUC SCORE
0.8284759358288771


In [ ]:
# gradient boosting 성능 확인
from sklearn.ensemble import GradientBoostingClassifier

gb_temp = GradientBoostingClassifier(max_depth = 2)
gb_temp.fit(X_train, y_train)
prediction6 = gb_temp.predict(X_val)

print('AUC SCORE')
print(roc_auc_score(y_val, prediction6))

AUC SCORE
0.8074866310160428


In [ ]:
# decision tree predict
prediction_dt = dt_temp.predict(test7)

In [ ]:
# decision tree submission
submission_dt = submission
submission_dt['Survived'] = prediction_dt
submission_dt.to_csv('submission_dt.csv', index = False)

In [ ]:
# logistic regression predict
prediction_logreg = logreg_temp.predict(test7)

In [ ]:
# logistic regression submission
submission_logreg = submission
submission_logreg['Survived'] = prediction_logreg
submission_logreg.to_csv('submission_logreg.csv', index = False)

In [ ]:
# xgboost predict
prediction_xgb = xgb_temp.predict(test7)

In [ ]:
# xgboost submission
submission_xgb = submission
submission_xgb['Survived'] = prediction_xgb
submission_xgb.to_csv('submission_xgb.csv', index = False)

In [ ]:
# gradient boosting predict
prediction_gb = gb_temp.predict(test7)

In [ ]:
# gradient boosting submission
submission_gb = submission
submission_gb['Survived'] = prediction_gb
submission_xgb.to_csv('submission_gb.csv', index = False)